# CycleGAN

To build a CycleGAN with PyTorch, you'd typically follow these steps:

Define the Dataset and DataLoader: Use torch.utils.data.Dataset to custom load your images, applying necessary transformations like resizing, normalizing, and converting to tensors. DataLoader will handle batching and shuffling of the data for training.
Build Generator and Discriminator Models: Define your generator and discriminator classes inheriting from nn.Module, using convolutional layers (nn.Conv2d for discriminators, nn.ConvTranspose2d for generators), batch normalization (nn.BatchNorm2d), and activation functions (nn.ReLU, nn.Tanh).
Specify Loss Functions and Optimizers: Commonly, CycleGAN uses adversarial loss (BCE loss) and cycle consistency loss (L1 loss), with optimizers typically being Adam.
Training Loop: Implement the training process, alternating between training the discriminator and generator, applying the gradients, and updating the network weights.
Visualization and Evaluation: Use matplotlib.pyplot and torchvision.utils for visualizing training progress and results.

# Dataset Preparation

In [8]:
import pathlib
import zipfile
import os
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import numpy as np
from PIL import Image
import shutil
import itertools
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.utils import make_grid

class ImageDataset(Dataset):
    def __init__(self, root_monet, root_photo, transform=None):
        self.transform = transform
        self.files_monet = [os.path.join(root_monet, file) for file in sorted(os.listdir(root_monet))]
        self.files_photo = [os.path.join(root_photo, file) for file in sorted(os.listdir(root_photo))]

    def __len__(self):
        return max(len(self.files_monet), len(self.files_photo))

    
    def __getitem__(self, index):
        image_monet_path = self.files_monet[index % len(self.files_monet)]
        image_photo_path = self.files_photo[index % len(self.files_photo)]

        image_monet = Image.open(image_monet_path)
        image_photo = Image.open(image_photo_path)

        if self.transform:
            image_monet = self.transform(image_monet)
            image_photo = self.transform(image_photo)

        return {'Monet': image_monet, 'Photo': image_photo}


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageDataset(root_monet='monet_jpg', root_photo='photo_jpg', transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


# Generator and Discriminator Models

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [10]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.model(x)
        return x


# Training Loop

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [14]:
generator = Generator()
discriminator = Discriminator()

optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
criterion = nn.BCELoss()

num_epochs = 50  

for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        real_images = batch['Photo'].to(device)  
        fake_images = generator(real_images).detach()
        real_loss = criterion(discriminator(real_images), torch.ones_like(discriminator(real_images)))
        fake_loss = criterion(discriminator(fake_images), torch.zeros_like(discriminator(fake_images)))
        d_loss = (real_loss + fake_loss) / 2
        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        fake_images = generator(real_images)
        g_loss = criterion(discriminator(fake_images), torch.ones_like(discriminator(fake_images)))
        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()
        
        if i % 100 == 0:  
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(dataloader)}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

Epoch [0/50], Step [0/7038], D Loss: 0.7154467105865479, G Loss: 0.7505959868431091
Epoch [0/50], Step [100/7038], D Loss: 0.5808931589126587, G Loss: 0.8234001994132996
Epoch [0/50], Step [200/7038], D Loss: 0.6623374223709106, G Loss: 0.8591057062149048
Epoch [0/50], Step [300/7038], D Loss: 0.6844042539596558, G Loss: 0.7739900946617126
Epoch [0/50], Step [400/7038], D Loss: 0.6526572704315186, G Loss: 0.8366765379905701
Epoch [0/50], Step [500/7038], D Loss: 0.5689939260482788, G Loss: 0.9517291188240051
Epoch [0/50], Step [600/7038], D Loss: 0.6554127931594849, G Loss: 0.8152122497558594
Epoch [0/50], Step [700/7038], D Loss: 0.6952806711196899, G Loss: 0.7639557123184204
Epoch [0/50], Step [800/7038], D Loss: 0.6154552698135376, G Loss: 0.8222744464874268
Epoch [0/50], Step [900/7038], D Loss: 0.5528801083564758, G Loss: 0.8823978304862976
Epoch [0/50], Step [1000/7038], D Loss: 0.5365228652954102, G Loss: 1.0472447872161865
Epoch [0/50], Step [1100/7038], D Loss: 0.4942954778671

# Visualization

In [15]:
def show_generated_images(images, nmax=5):
    fig, axes = plt.subplots(figsize=(10, 5), nrows=1, ncols=nmax, sharey=True, sharex=True)
    for ax, img in zip(axes.flatten(), images[:nmax]):
        img = img.detach().cpu().numpy().transpose(1, 2, 0)
        img = (img + 1) / 2 
        ax.imshow(img)
        ax.axis('off')
    plt.show()


# Submission

In [ ]:
import shutil
from torchvision.utils import save_image
import torch

generator.eval()
save_dir = './tmp/monet_imgs'
os.makedirs(save_dir, exist_ok=True)

for i in range(7000):
    with torch.no_grad():
        noise = torch.randn(1, 100, 1, 1, device=device)
        generated_image = generator(noise)
        generated_image = (generated_image + 1) / 2  
        save_path = f'{save_dir}/{i:04d}.jpg'
        save_image(generated_image.cpu(), save_path)
shutil.make_archive('images', 'zip', save_dir)

shutil.move('images.zip', './images.zip')

shutil.rmtree(save_dir)

# Conclusions

This workflow demonstrates a practical application of deep learning models for image generation, showcasing not only the model's inference capabilities but also the necessary post-processing steps for handling and distributing the generated data. The use of PyTorch for model inference and standard Python libraries for file management underscores the importance of integrating deep learning models with broader software engineering practices to create end-to-end pipelines.

The provided adjustments ensure that the code runs efficiently and correctly, addressing common issues such as device management for tensors, data normalization for image visualization, and robust file handling. Successfully executing this pipeline can serve as a foundation for more complex projects involving image generation, such as creating datasets for further machine learning projects, digital art creation, or enhancing data augmentation strategies for training other models.